In [5]:
# Import dependencies
import pandas as pd

In [6]:
# Reference to CSV and reading CSV into Pandas DataFrame
# csv_path1 = "Resources/country_vaccinations.csv"
# csv_path2 = "Resources/gdp-csv.csv"
# csv_path3 = "Resources/time_series_covid_19_confirmed.csv"
# csv_path4 = "Resources/time_series_covid_19_deaths.csv"
csv_path5 = "Resources/time_series_covid_19_recovered.csv"
# csv_path6 = "Resources/WorldPopulation.csv"
# csv_path7 = "Resources/country_vaccinations_by_manufacturer.csv"

# merging final combined CSV meta file

# country_vaccinations_df = pd.read_csv(csv_path1)
# gdp_df = pd.read_csv(csv_path2)
# covid_19_confirmed_df = pd.read_csv(csv_path3)
# covid_19_deaths_df = pd.read_csv(csv_path4)
covid_19_recovered_df = pd.read_csv(csv_path5)
# WorldPopulations_df = pd.read_csv(csv_path6)
# country_vaccinations_by_manufacturer_df = pd.read_csv(csv_path7)



In [7]:
covid_19_recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,52363,52392,52489,52520,52794,52974,53103,53206,53272,53391
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,104278,105016,105728,106465,107163,107887,108606,109338,110172,110920
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,84038,84167,84299,84440,84598,84745,84940,85108,85249,85398
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,12423,12458,12491,12519,12561,12590,12650,12684,12684,12753
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,23089,23092,23092,23288,23341,23584,23606,23876,23913,23913


In [8]:
covid_19_recovered_df['Country/Region']

0             Afghanistan
1                 Albania
2                 Algeria
3                 Andorra
4                  Angola
              ...        
255               Vietnam
256    West Bank and Gaza
257                 Yemen
258                Zambia
259              Zimbabwe
Name: Country/Region, Length: 260, dtype: object

In [9]:
covid_19_recovered_df['Country/Region'].nunique()

192

In [10]:
covid_19_recovered_df['Country/Region'].count()

260

In [11]:
covid_19_recovered_df['Country/Region'].nunique

<bound method IndexOpsMixin.nunique of 0             Afghanistan
1                 Albania
2                 Algeria
3                 Andorra
4                  Angola
              ...        
255               Vietnam
256    West Bank and Gaza
257                 Yemen
258                Zambia
259              Zimbabwe
Name: Country/Region, Length: 260, dtype: object>

In [17]:
recovered = covid_19_recovered_df

recovered.columns[4:]

recovered_long = recovered.melt(id_vars = ['Province/State','Country/Region','Lat','Long'],
                         value_vars = recovered.columns[4:],
                         var_name = 'Dates',  
                         value_name='Recovered')
del recovered_long['Province/State']
recovered_long['Dates'] = pd.to_datetime(recovered_long['Dates'])
recovered_long['Month'] = recovered_long['Dates'].dt.month
recovered_long['Year'] = recovered_long['Dates'].dt.year
del recovered_long['Dates']


del recovered_long['Lat']
del recovered_long['Long']

recovered_long_clean = recovered_long.sort_values(['Month','Year'])
recovered_long_clean

,Country/Region,Recovered,Month,Year
0,Afghanistan,0,1,2020
1,Albania,0,1,2020
2,Algeria,0,1,2020
3,Andorra,0,1,2020
4,Angola,0,1,2020
...,...,...,...,...
89695,Vietnam,1325,12,2020
89696,West Bank and Gaza,117183,12,2020
89697,Yemen,1394,12,2020
89698,Zambia,18660,12,2020


In [21]:

recovered_long_date = recovered_long_clean.groupby(['Month','Year'])['recovered'].sum().reset_index()
recovered_long_date = recovered_long_date.sort_values(['Month','Year'])

recovered_2020 = recovered_long_date[recovered_long_date['Year'] == 2020]
recovered_2020['Date'] = recovered_2020['Month'].astype(str)+ ' / ' + recovered_2020['Year'].astype(str)
del recovered_2020['Month']
del recovered_2020['Year']

recovered_2021 = recovered_long_date[recovered_long_date['Year'] == 2021]
recovered_2021['Date'] = recovered_2021['Month'].astype(str)+ ' / ' + recovered_2021['Year'].astype(str)
del recovered_2021['Month']
del recovered_2021['Year']

recovered_total_per_month = recovered_2020.append(recovered_2021)

KeyError: 'Column not found: recovered'

In [18]:
recovered_total_per_month = recovered_total_per_month[['Date','Recovered']].reset_index()
del recovered_total_per_month['index']
recovered_total_per_month.to_csv('recovered_date.csv')
print(recovered_total_per_month)

KeyError: "['Recovered'] not in index"

In [19]:

recovered_long_country = recovered_long_clean.groupby('Country/Region').sum()
del recovered_long_country['Month']
del recovered_long_country['Year']
recovered_long_country.to_csv('recovered_country.csv')
print(recovered_long_country)

                    Recovered
Country/Region               
Afghanistan          11975357
Albania              10629990
Algeria              16599409
Andorra               1807691
Angola                2984212
...                       ...
Vietnam                413898
West Bank and Gaza   29139545
Yemen                  422750
Zambia               10174297
Zimbabwe              4615427

[192 rows x 1 columns]
